In [1]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [2]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [3]:
img_folder = r'D:\original'
out_folder = r'D:\ToAWS2'

course_list = ['청도','포항','중문','시흥','거창','속리산','춘천']

course_ids = {
  '포항':'MGC001',
  '청도':'MGC002',
  '중문':'MGC003',
  '시흥':'MGC004',
  '거창':'MGC005',
  '속리산':'MGC006',
  '춘천':'MGC007'
}

folder_type = ['rgb','ndvi','thumb']

In [4]:

course_id = course_ids.get('시흥')
course_id

'MGC004'

In [5]:
folders_candidate =glob.glob(os.path.join(out_folder,course_id,'*'))
folders_candidate

['D:\\ToAWS2\\MGC004\\20230807']

In [6]:
target_folder = folders_candidate[0]
target_folder

'D:\\ToAWS2\\MGC004\\20230807'

In [7]:
target_date = os.path.split(target_folder)[-1]
target_date 

'20230807'

Gather all data json files from Output Folder

In [11]:
dataJsonList = glob.glob(os.path.join(out_folder,course_id,target_date,'**/data*.json'), recursive=True)
dataJsonList

['D:\\ToAWS2\\MGC004\\20230807\\아세코밸리_1H\\data20230807아세코밸리_1H.json',
 'D:\\ToAWS2\\MGC004\\20230807\\아세코밸리_5H\\data20230807아세코밸리_5H.json',
 'D:\\ToAWS2\\MGC004\\20230807\\아세코밸리_8H\\data20230807아세코밸리_8H.json',
 'D:\\ToAWS2\\MGC004\\20230807\\아세코밸리_9H\\data20230807아세코밸리_9H.json']

In [13]:
folderinDataJson = []
totalPhotoJson = []

for dataJson_ in dataJsonList:
  with open(dataJson_, "r", encoding='utf-8') as _json:
    dataJson= json.load(_json)


  for data_ in dataJson:
    # if data_['dest']['thumb'][0] !="M" :
      data_['dest']['thumb'] = os.path.join(course_id,target_date,data_['info']['area'],data_['info']['desc'],'thumb','thumbrgb{}.JPG'.format( data_['id']) )
    # if data_['dest']['rgb'][0] !="M" :
      data_['dest']['rgb'] = os.path.join(course_id,target_date,data_['info']['area'],data_['info']['desc'],'rgb','rgb{}.JPG'.format( data_['id']) )
    # if data_['dest']['ndvi'][0] !="M" :
      data_['dest']['ndvi'] = os.path.join(course_id,target_date,data_['info']['area'],data_['info']['desc'],'ndvi','ndvi{}.JPG'.format( data_['id']) )
    
  
  totalPhotoJson.extend(dataJson)

  for Json_ in dataJson:
    if Json_['destFolder'] not in folderinDataJson:
      folderinDataJson.append(Json_['destFolder'])




In [14]:
len(totalPhotoJson)

5787

In [15]:
totalPhotoJson[0]

{'id': '20230807111124_1267705611389_373679901111_57127000',
 'originalFileJPG': 'NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0807_시흥 아세코밸리_맑음_데이터 1차 분류본_완\\아세코밸리_1H\\2023_0807_시흥 아세코밸리_1H_0.5m\\20230807_1111-1114_잡초,이종잔디,벤트\\DJI_20230807111124_0073_D.JPG',
 'originalFileGrp': ['NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0807_시흥 아세코밸리_맑음_데이터 1차 분류본_완\\아세코밸리_1H\\2023_0807_시흥 아세코밸리_1H_0.5m\\20230807_1111-1114_잡초,이종잔디,벤트\\DJI_20230807111124_0073_MS_G.TIF',
  'NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0807_시흥 아세코밸리_맑음_데이터 1차 분류본_완\\아세코밸리_1H\\2023_0807_시흥 아세코밸리_1H_0.5m\\20230807_1111-1114_잡초,이종잔디,벤트\\DJI_20230807111124_0073_MS_NIR.TIF',
  'NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0807_시흥 아세코밸리_맑음_데이터 1차 분류본_완\\아세코밸리_1H\\2023_0807_시흥 아세코밸리_1H_0.5m\\20230807_1111-1114_잡초,이종잔디,벤트\\DJI_20230807111124_0073_MS_R.TIF',
  'NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0807_시흥 아세코밸리_맑음_데이터 1차 분류본_완\\아세코밸리_1H\\2023_0807_시흥 아세코밸리_1H_0.5m\\20230807_1111-1114_잡초,이종잔디,벤트\\DJI_20230807111124_0073_MS_RE.TIF'],
 'destFolde

In [39]:
save_name = os.path.join(out_folder,course_id,target_date, 'photo.json')
with open(save_name, "w", encoding='utf-8') as final:
   json.dump(totalPhotoJson, final , ensure_ascii=False)

In [40]:
filestructure = []


for folder_ in folderinDataJson:
  fileinfo = {}
  fileinfo["path"] = folder_
  fileinfo["file_cnt"] = len([ x for x in totalPhotoJson if x["destFolder"] == folder_])
  fileinfo["labeled_file_cnt"] = 0
  filestructure.append(fileinfo)
  

In [41]:
save_name = os.path.join(out_folder,course_id,target_date, 'filepath_'+course_id + '_'+target_date+'.json')
with open(save_name, "w", encoding='utf-8') as final:
   json.dump(filestructure, final , ensure_ascii=False)